In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
%pip install scarches scanpy anndata muon gdown typing_extensions | grep -v 'already satisfied'

  Obtaining dependency information for scarches from https://files.pythonhosted.org/packages/c4/c8/3a9db7d797e4db91cfc96971cfd4a42c86e9cdbcb56416c1de8d3c534bba/scArches-0.5.10-py3-none-any.whl.metadata
  Obtaining dependency information for scanpy from https://files.pythonhosted.org/packages/39/d5/992cb22882523b873b981c1bdebbf28baffeb7ac931cd5d60cb53f2d8a24/scanpy-1.9.6-py3-none-any.whl.metadata
  Obtaining dependency information for anndata from https://files.pythonhosted.org/packages/8a/87/201514af3bf08db52e11b7d94e6129f0a75503194b81614ff48883101c4c/anndata-0.10.3-py3-none-any.whl.metadata
  Obtaining dependency information for muon from https://files.pythonhosted.org/packages/8e/1a/e12b1d6a9efea45f4bb99bd6be33fff645314c6479d5fb33c5683391b015/muon-0.1.5-py3-none-any.whl.metadata
  Obtaining dependency information for scHPL>=1.0.0 from https://files.pythonhosted.org/packages/71/fe/9d8b999436e7192b2303650d9e8f592124f51144479173e41673f99bb1ec/scHPL-1.0.4-py3-none-any.whl.metadata
  Obta

In [3]:
!curl -O https://covid19.cog.sanger.ac.uk/submissions/release1/haniffa21.processed.h5ad

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6854M  100 6854M    0     0   153M      0  0:00:44  0:00:44 --:--:-- 98.1M00:06  0:00:27  209M


In [4]:
import scarches as sca
import scanpy as sc
import anndata as ad
import numpy as np
import muon
import gdown
import json

/opt/conda/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/opt/conda/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
2023-12-10 20:11:43,265	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-12-10 20:11:43,914	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
data = sc.read('haniffa21.processed.h5ad')
data

/opt/conda/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


AnnData object with n_obs × n_vars = 647366 × 24929
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [15]:
def explore_obs(tag):
    values, counts = np.unique(data.obs[tag], return_counts=True)
    print(f"{len(values)} unique {tag} elements")
    print(f"--------------------------------------------")
    
    max_value_len = max([len(value) for value in values])
    max_count_len = max([len(str(count)) for count in counts])
    
    for i, (value, count) in enumerate(reversed(sorted(list(zip(values, counts)), key = lambda x: x[1]))):
        print(f"{value:<{max_value_len+3}} {count:>{max_count_len}}")

In [16]:
explore_obs("patient_id")

130 unique patient_id elements
--------------------------------------------
MH9143427         14317
AP6               14086
MH8919333         12081
MH9143277         11710
AP11              10921
MH9143326         10446
MH9143327         10295
MH9143276         10207
MH9143270         10084
MH9143323         10082
MH9143324         10014
MH9143422          9608
MH9143271          9542
MH9143272          9458
MH9143423          9083
MH9143325          9019
MH9179825          8919
MH8919327          8819
MH9143321          8639
AP12               8574
MH9143424          8530
AP5                8511
MH9143275          8371
MH9143322          8244
newcastle49        8232
MH9179824          8205
MH8919328          8182
MH9143421          8111
MH8919332          8004
MH9143273          7987
MH9143420          7985
MH9143426          7953
MH8919326          7816
MH8919330          7605
MH8919331          7585
AP4                7446
newcastle21v2      7300
MH9143320          7128
MH8919283   

In [17]:
explore_obs('initial_clustering')

18 unique initial_clustering elements
--------------------------------------------
CD4             146380
CD14            121283
CD8             101704
NK_16hi          95311
B_cell           66833
CD16             20403
Platelets        17712
Treg             13256
NK_56hi          11672
gdT              10634
Plasmablast       9915
Lymph_prolif      7068
DCs               6938
MAIT              6835
pDC               4778
HSC               3325
RBC               2672
Mono_prolif        647


In [19]:
explore_obs('Worst_Clinical_Status')

10 unique Worst_Clinical_Status elements
--------------------------------------------
Moderate        179012
Healthy          97039
Mild             93835
nan              74913
Critical         63854
Death            41836
Severe           40235
Asymptomatic     33601
Non-covid        15157
LPS               7884


In [18]:
data.X.shape

(647366, 24929)